In [1]:
import numpy as np
import pandas as pd
import re
import datetime as dt

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import string

import nltk 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold


from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')
import matplotlib.patches as mpatches

In [2]:
train = pd.read_csv('/Users/kimginam/Data176/NLP/소설작가분류/open/train.csv')
test = pd.read_csv('/Users/kimginam/Data176/NLP/소설작가분류/open/test_x.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kimginam/Data176/NLP/소설작가분류/open/train.csv'

In [ ]:
import joblib

In [ ]:
import pickle

Encoding : TF-IDF<br>
modeling : randomforest

In [13]:
test.head(2)

,index,text,text_new
0,0,“Not at all. I think she is one of the most ch...,think one charming young ladies ever met might...
1,1,"""No,"" replied he, with sudden consciousness, ""...",replied sudden consciousness find cannot ignor...


In [16]:
train

,index,text,author,text_new
0,0,"He was almost choking. There was so much, so m...",3,almost choking much much wanted say strange ex...
1,1,"“Your sister asked for it, I suppose?”",2,sister asked suppose
2,2,"She was engaged one day as she walked, in per...",1,engaged one day walked perusing janes last let...
3,3,"The captain was in the porch, keeping himself ...",4,captain porch keeping carefully way treacherou...
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3,mercy gentlemen odin flung hands dont write an...
...,...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2,mr smith odin whispered hardly dared hope woul...
54875,54875,"I told my plan to the captain, and between us ...",4,told plan captain us settled details accomplis...
54876,54876,"""Your sincere well-wisher, friend, and sister...",1,sincere wellwisher friend sister lucy odin
54877,54877,“Then you wanted me to lend you money?”,3,wanted lend money


In [14]:
train.head(2)

,index,text,author,text_new
0,0,"He was almost choking. There was so much, so m...",3,almost choking much much wanted say strange ex...
1,1,"“Your sister asked for it, I suppose?”",2,sister asked suppose


In [8]:
#Remove Urls and HTML links
def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)
train['text_new']=train['text'].apply(lambda x:remove_urls(x))
test['text_new']=test['text'].apply(lambda x:remove_urls(x))

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
train['text_new']=train['text_new'].apply(lambda x:remove_html(x))
test['text_new']=test['text_new'].apply(lambda x:remove_html(x))

In [9]:
# Lower casing
def lower(text):
    low_text= text.lower()
    return low_text
train['text_new']=train['text_new'].apply(lambda x:lower(x))
test['text_new']=test['text_new'].apply(lambda x:lower(x))


# Number removal
def remove_num(text):
    remove= re.sub(r'\d+', '', text)
    return remove
train['text_new']=train['text_new'].apply(lambda x:remove_num(x))
test['text_new']=test['text_new'].apply(lambda x:remove_num(x))

In [10]:
#Remove stopwords & Punctuations
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def punct_remove(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    return punct
train['text_new']=train['text_new'].apply(lambda x:punct_remove(x))
test['text_new']=test['text_new'].apply(lambda x:punct_remove(x))


def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
train['text_new']=train['text_new'].apply(lambda x:remove_stopwords(x))
test['text_new']=test['text_new'].apply(lambda x:remove_stopwords(x))

In [13]:
train.head(2)

,index,text,author,text_new
0,0,"He was almost choking. There was so much, so m...",3,almost choking much much wanted say strange ex...
1,1,"“Your sister asked for it, I suppose?”",2,sister asked suppose


In [14]:
test.head(2)

,index,text,text_new
0,0,“Not at all. I think she is one of the most ch...,think one charming young ladies ever met might...
1,1,"""No,"" replied he, with sudden consciousness, ""...",replied sudden consciousness find cannot ignor...


In [24]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import StratifiedKFold

In [12]:
target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [17]:
# vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
# X_cnt = vec.fit_transform(train['text_new'])
# print(X_cnt.shape)

In [16]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X = vec.fit_transform(train['text_new'])
X_tst = vec.transform(test['text_new'])
print(X.shape, X_tst.shape)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(54879, 3355) (19617, 3355)


In [19]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [21]:
y = train.author.values
y.shape

(54879,)

In [22]:
p = np.zeros((X.shape[0], n_class))
p_tst = np.zeros((X_tst.shape[0], n_class))
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
    clf = LogisticRegression()
    clf.fit(X[i_trn], y[i_trn])
    p[i_val, :] = clf.predict_proba(X[i_val])
    p_tst += clf.predict_proba(X_tst) / n_class

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [28]:
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

Log Loss (CV):   0.8561


In [67]:
sub = pd.read_csv('/Users/kimginam/Data176/NLP/소설작가분류/open/sample_submission.csv',index_col=0)

In [68]:
print(sub.shape)
sub.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [69]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2,3,4
index,,,,,
0,0.042951,0.697204,0.150134,0.089203,0.020507
1,0.234962,0.228165,0.119533,0.108933,0.308407
2,0.842447,0.021445,0.049168,0.009474,0.077465
3,0.102822,0.003485,0.640939,0.004690,0.248064
4,0.298971,0.246439,0.080226,0.189603,0.184761


In [70]:
sub.to_csv('/Users/kimginam/Data176/NLP/소설작가분류/open/submission.csv')

#  feature enginnering
갯수 컬럼 추가<br>
문장이나 단어 갯수 카운트 컬럼 추가<br>
특수문자 갯수 컬럼 추가<br>
작가별 평균 단어수

In [75]:
# To remove the special cha
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def remove_num(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

# Applying helper functions

train['text_clean'] = train['text'].apply(lambda x: remove_URL(x))
train['text_clean'] = train['text_clean'].apply(lambda x: remove_emoji(x))
train['text_clean'] = train['text_clean'].apply(lambda x: remove_html(x))
train['text_clean'] = train['text_clean'].apply(lambda x: remove_punct(x))
train['text_clean'] = train['text_clean'].apply(lambda x: remove_num(x))

train['text_clean'] = train['text_clean'].str.replace('“','')
train['text_clean'] = train['text_clean'].str.replace('”','')
train['text_clean'] = train['text_clean'].str.replace('"','')

In [76]:
train['word_count'] = train['text_clean'].apply(lambda x: len(str(x).split()))

In [78]:
train[train['author'] == 0]['word_count'].mean()

37.40204004533434

In [79]:
train[train['author'] == 1]['word_count'].mean()

57.11284962614234

In [80]:
train[train['author'] == 2]['word_count'].mean()

40.07460619698806

In [81]:
train[train['author'] == 3]['word_count'].mean()

41.00092942972847

In [82]:
train[train['author'] == 4]['word_count'].mean()

41.555285073670724

In [83]:
train.loc[(train['author'] == 0), 'word_avg'] = 37.4
train.loc[(train['author'] == 1), 'word_avg'] = 57.1
train.loc[(train['author'] == 2), 'word_avg'] = 40
train.loc[(train['author'] == 3), 'word_avg'] = 41
train.loc[(train['author'] == 4), 'word_avg'] = 41.5

In [84]:
au0 = train[train['author'] == 0]
au1 = train[train['author'] == 1]
au2 = train[train['author'] == 2]
au3 = train[train['author'] == 3]
au4 = train[train['author'] == 4]

In [85]:
train.head(5)

,index,text,author,text_clean,word_count,word_avg
0,0,"He was almost choking. There was so much, so m...",3,He was almost choking There was so much so muc...,46,41.0
1,1,"“Your sister asked for it, I suppose?”",2,Your sister asked for it I suppose,7,40.0
2,2,"She was engaged one day as she walked, in per...",1,She was engaged one day as she walked in peru...,57,57.1
3,3,"The captain was in the porch, keeping himself ...",4,The captain was in the porch keeping himself c...,58,41.5
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3,Have mercy gentlemen odin flung up his hands D...,39,41.0


count special characters

In [86]:
def extract_special_cha(text):
    return re.sub('[\w]+' ,'', text)

In [87]:
train['special_cha'] = train['text'].apply(lambda x : extract_special_cha(x))

In [88]:
# To remove white splace
train['special_cha'] = train['special_cha'].replace(" ",'',regex=True)

In [89]:
def count_special_cha(text):
    return len(re.sub('[\w]+' ,'', text))

train['count_special_cha'] = train['special_cha'].apply(lambda x: count_special_cha(x))

In [98]:
train.head(20)

,index,text,author,text_clean,word_count,word_avg,special_cha,count_special_cha,count_quote,count_quote2
0,0,"He was almost choking. There was so much, so m...",3,He was almost choking There was so much so muc...,46,41.0,".,,.,;,.",8,0,0
1,1,"“Your sister asked for it, I suppose?”",2,Your sister asked for it I suppose,7,40.0,"“,?”",4,2,2
2,2,"She was engaged one day as she walked, in per...",1,She was engaged one day as she walked in peru...,57,57.1,",’,,,.,.,:",10,0,0
3,3,"The captain was in the porch, keeping himself ...",4,The captain was in the porch keeping himself c...,58,41.5,",,.,“'..,;,;,.,.,,.”",20,2,2
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3,Have mercy gentlemen odin flung up his hands D...,39,41.0,"“,!”.“’,;.’,....,!”",19,4,4
5,5,"""It was well fought,"" he said, ""and, by my soo...",4,It was well fought he said and by my sooth the...,16,41.5,""","","",,.""",9,4,0
6,6,"Not to pay him was impossible, considering his...",3,Not to pay him was impossible considering his ...,23,41.0,",.,,.",5,0,0
7,7,"“A proper figure of a man at-arms,” said the l...",2,A proper figure of a man atarms said the littl...,77,40.0,"“-,”.“,,,...,,.”",16,4,4
8,8,"'You were not here last Sunday night,' he said.",0,You were not here last Sunday night he said,9,37.4,"','.",4,0,0
9,9,“You must not ask me that!” I cried. “Hell may...,4,You must not ask me that I cried Hell may have...,32,41.5,"“!”.“.,,,.”",11,4,4


count double quotes

In [94]:
train['count_quote'] = train['text'].apply(lambda x : x.count('"')) + train['text'].apply(lambda x : x.count('”')) + train['text'].apply(lambda x : x.count('“'))